In [1]:
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline

import spacy

In [2]:
data = pd.read_csv("E:\Coding\Python\AI\PreProd\Week4DIY\DIY_TransferLearning\Data\TwitterData/twitter_data.csv")

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75682 entries, 0 to 75681
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   tweetID        75682 non-null  int64 
 1   entity         75682 non-null  object
 2   sentiment      75682 non-null  object
 3   tweet_content  74996 non-null  object
dtypes: int64(1), object(3)
memory usage: 2.3+ MB


In [4]:
data.head()

,tweetID,entity,sentiment,tweet_content
0,2401,Borderlands,Positive,im getting on borderlands and i will murder yo...
1,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
2,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
3,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
4,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...


In [5]:
data.shape

(75682, 4)

In [6]:
data.drop_duplicates(inplace=True)
data.dropna(inplace=True)

In [7]:
nlp = spacy.load("en_core_web_sm") 
def preprocess(text):
    doc = nlp(text)
    filtered_tokens = []
    for token in doc:
        if token.is_stop or token.is_punct:
            continue
        filtered_tokens.append(token.lemma_)
    
    return " ".join(filtered_tokens) 

In [8]:
data['Preprocessed Text'] = data['tweet_content'].apply(preprocess) 
    # Encode categorical columns
le = LabelEncoder() # Initialize the LabelEncoder
data['sentiment'] = le.fit_transform(data['sentiment'])

In [9]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments, TFBertForSequenceClassification
from datasets import Dataset
import tensorflow as tf

In [10]:
data['Preprocessed Text'] = data['Preprocessed Text'].astype(str)  # Convert all to string
data = data[data['Preprocessed Text'].str.strip() != '']  # Drop empty strings

In [11]:
data = data.groupby('sentiment', group_keys=False).apply(lambda x: x.sample(frac=5000/75682, random_state=42))
X = data["Preprocessed Text"]
y = data["sentiment"]

C:\Users\prach\AppData\Local\Temp\ipykernel_15732\2789582200.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  data = data.groupby('sentiment', group_keys=False).apply(lambda x: x.sample(frac=5000/75682, random_state=42))


In [12]:
from sklearn.model_selection import train_test_split
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)

# Split the temporary set into validation (50%) and test (50%) to get 10% each
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

In [13]:
print(f"Length of X_train: {len(X_train)}, Length of y_train: {len(y_train)}")
print(f"Length of X_val: {len(X_val)}, Length of y_val: {len(y_val)}")
print(f"Length of X_test: {len(X_test)}, Length of y_test: {len(y_test)}")


Length of X_train: 3724, Length of y_train: 3724
Length of X_val: 466, Length of y_val: 466
Length of X_test: 466, Length of y_test: 466


In [14]:
#Tokenize and encode the data using the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)


In [15]:
max_len= 128
# Tokenize and encode the sentences
X_train_encoded = tokenizer.batch_encode_plus(X.tolist(),
											padding=True, 
											truncation=True,
											max_length = max_len,
											return_tensors='tf')

X_val_encoded = tokenizer.batch_encode_plus(X_val.tolist(), 
											padding=True, 
											truncation=True,
											max_length = max_len,
											return_tensors='tf')

X_test_encoded = tokenizer.batch_encode_plus(X_test.tolist(), 
											padding=True, 
											truncation=True,
											max_length = max_len,
											return_tensors='tf')


In [16]:
# Intialize the model
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=4)


All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [17]:
# Compile the model with an appropriate optimizer, loss function, and metrics
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
model.compile(optimizer=optimizer, loss=loss, metrics=[metric])


In [18]:
# Step 5: Train the model
history = model.fit(
	[X_train_encoded['input_ids'], X_train_encoded['token_type_ids'], X_train_encoded['attention_mask']],
	y,
	validation_data=(
	[X_val_encoded['input_ids'], X_val_encoded['token_type_ids'], X_val_encoded['attention_mask']],y_val),
	batch_size=32,
	epochs=3
)


Epoch 1/3
146/146 [==============================] - 1601s 11s/step - loss: 1.3649 - accuracy: 0.3080 - val_loss: 1.3430 - val_accuracy: 0.3648
Epoch 2/3
146/146 [==============================] - 1351s 9s/step - loss: 1.3233 - accuracy: 0.3733 - val_loss: 1.2627 - val_accuracy: 0.4549
Epoch 3/3
146/146 [==============================] - 1364s 9s/step - loss: 1.2129 - accuracy: 0.4901 - val_loss: 1.2059 - val_accuracy: 0.5172


In [35]:
#Evaluate the model on the test data
test_loss, test_accuracy = model.evaluate(
	[X_test_encoded['input_ids'], X_test_encoded['token_type_ids'], X_test_encoded['attention_mask']],
	y_test
)

train_accuracy = history.history['accuracy'][-1]  # Last training accuracy
val_accuracy = history.history['val_accuracy'][-1]  # Last validation accuracy

# Print the results
print(f'Test loss: {test_loss}, Test accuracy: {test_accuracy}')
print(f'Training accuracy: {train_accuracy}, Validation accuracy: {val_accuracy}')



15/15 [==============================] - 38s 2s/step - loss: 1.1353 - accuracy: 0.5236
Test loss: 1.1353317499160767, Test accuracy: 0.5236051678657532
Training accuracy: 0.4901202619075775, Validation accuracy: 0.5171673893928528


In [31]:
from sklearn.metrics import classification_report, precision_score, recall_score, f1_score
logits = model.predict([X_test_encoded['input_ids'], X_test_encoded['token_type_ids'], X_test_encoded['attention_mask']]).logits

15/15 [==============================] - 33s 2s/step


In [34]:

y_pred_probs = tf.nn.softmax(logits, axis=-1).numpy()

# If it's binary classification, use a threshold of 0.5 on the positive class probability
if y_pred_probs.shape[1] == 2:
    y_pred = (y_pred_probs[:, 1] > 0.5).astype(int)
else:
    # For multi-class classification, take the argmax of the probabilities
    y_pred = np.argmax(y_pred_probs, axis=1)
    
y_test = np.array(y_test)

print(f"Precision: {precision_score(y_test, y_pred, average='weighted')}")
print(f"Recall: {recall_score(y_test, y_pred, average='weighted')}")
print(f"F1 Score: {f1_score(y_test, y_pred, average='weighted')}")

Precision: 0.5771451838762672
Recall: 0.5236051502145923
F1 Score: 0.47864764317974173


In [20]:
# model_name = "bert-base-uncased"
# tokenizer = BertTokenizer.from_pretrained(model_name)
# model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)

In [21]:
# Combine X and y back into DataFrames for each split
# train_df = pd.DataFrame({'Preprocessed text': X_train.to_numpy(), 'sentiment': y_train.to_numpy()})
# val_df = pd.DataFrame({'Preprocessed text': X_val.to_numpy(), 'sentiment': y_val.to_numpy()})
# test_df = pd.DataFrame({'Preprocessed text': X_test.to_numpy(), 'sentiment': y_test.to_numpy()})

# # Convert pandas DataFrames to Hugging Face Dataset objects
# train_dataset = Dataset.from_pandas(train_df)
# val_dataset = Dataset.from_pandas(val_df)
# test_dataset = Dataset.from_pandas(test_df)

In [22]:
# def tokenize_function(examples):
#     # Check if 'text' is a list, and tokenize accordingly
#     return tokenizer(examples["Preprocessed text"], padding="max_length", truncation=True)

# # Assuming 'data' is your dataset containing the text data
# train_dataset = train_dataset.map(tokenize_function, batched=True)
# test_dataset = test_dataset.map(tokenize_function, batched=True)
# val_dataset = val_dataset.map(tokenize_function, batched=True)

# # Set the format for PyTorch
# train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'sentiment'])
# test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'sentiment'])
# val_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'sentiment'])


In [23]:
# import transformers
# import accelerate
# import torch

# print("Transformers version:", transformers.__version__)
# print("Accelerate version:", accelerate.__version__)
# print("Torch version:", torch.__version__)



In [24]:
# training_args = TrainingArguments(
#     output_dir="./results",                 # Output directory for model predictions and checkpoints
#     evaluation_strategy="epoch",             # Evaluate after each epoch
#     learning_rate=2e-5,                      # Learning rate
#     per_device_train_batch_size=16,         # Batch size for training
#     per_device_eval_batch_size=16,          # Batch size for evaluation
#     num_train_epochs=3,                      # Total number of training epochs
#     weight_decay=0.01,                       # Weight decay for regularization
# )

# # Initialize the Trainer
# trainer = Trainer(
#     model=model,                             # Your model instance
#     args=training_args,                      # Training arguments
#     train_dataset=train_dataset,             # Training dataset
#     eval_dataset=val_dataset                 # Validation dataset
# )

# # Train the model
# trainer.train()